In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"


In [3]:

! wget -p data/ http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Arts_Crafts_and_Sewing.json.gz

--2020-02-25 17:06:08--  http://data/
Resolving data (data)... failed: No address associated with hostname.
wget: unable to resolve host address ‘data’
--2020-02-25 17:06:08--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Arts_Crafts_and_Sewing.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306424435 (292M) [application/octet-stream]
Saving to: ‘deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Arts_Crafts_and_Sewing.json.gz’

deepyeti.ucsd.edu/j 100%[===================>] 292.23M  11.0MB/s    in 28s     

2020-02-25 17:06:36 (10.5 MB/s) - ‘deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Arts_Crafts_and_Sewing.json.gz’ saved [306424435/306424435]

FINISHED --2020-02-25 17:06:36--
Total wall clock time: 29s
Downloaded: 1 files, 292M in 28s (10.5 MB/s)


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
pip install pyspark


     |████████████████████████████████| 217.8MB 56kB/s 
     |████████████████████████████████| 204kB 45.6MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=c429038e3ce5fba5935016b53a623e33bf5f1a38bccb6412119347620c2b260f
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [0]:
APP_NAME = "Exploratory_Data_Analysis"
SPARK_URL = "local[*]"


In [0]:
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()

In [0]:
data = spark.read.json('/content/drive/My Drive/Arts_Crafts_and_Sewing.json.gz') 


In [10]:
data.columns

['asin',
 'image',
 'overall',
 'reviewText',
 'reviewTime',
 'reviewerID',
 'reviewerName',
 'style',
 'summary',
 'unixReviewTime',
 'verified',
 'vote']

In [11]:
data.show()


+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|               style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|0449819906| null|    5.0|I've read this bo...| 11 6, 2013|A3U4E9PIZ8OWH1|      Shirley Curtis|[,,,,,,  Kindle E...|   A  WONDERFUL BOOK|    1383696000|    true|   4|
|0449819906| null|    5.0|Nicely written di...|02 28, 2017|A3945D2TJ0PI86|            M. Smith|[,,,,,,  Kindle E...|                Nice|    1488240000|    true|null|
|0449819906| null|    5.0|             love it|02 25, 2017|A2WZK72HLQ7SPT|     Amazon Customer|[,,,,,,  Paperbac...|          Five Stars|    1487980800|    true|null

In [12]:
data.printSchema()

root
 |-- asin: string (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: struct (nullable = true)
 |    |-- Color Name:: string (nullable = true)
 |    |-- Color:: string (nullable = true)
 |    |-- Design:: string (nullable = true)
 |    |-- Display Height:: string (nullable = true)
 |    |-- Edition:: string (nullable = true)
 |    |-- Flavor:: string (nullable = true)
 |    |-- Format:: string (nullable = true)
 |    |-- Item Display Length:: string (nullable = true)
 |    |-- Item Display Weight:: string (nullable = true)
 |    |-- Item Package Quantity:: string (nullable = true)
 |    |-- Length:: string (nullable = true)
 |    |-- Material Type:: string (nullable = true)
 |    |-- Material:: string (nullable = t

In [13]:
data.filter(" summary is null").show()

+----------+--------------------+-------+--------------------+-----------+--------------+-------------------+--------------------+-------+--------------+--------+----+
|      asin|               image|overall|          reviewText| reviewTime|    reviewerID|       reviewerName|               style|summary|unixReviewTime|verified|vote|
+----------+--------------------+-------+--------------------+-----------+--------------+-------------------+--------------------+-------+--------------+--------+----+
|B00004THXH|                null|    5.0|Just what I was l...|08 21, 2017|A2IHAQY2JV6WR7|                Pam|[,,,,,,,,,,,,,,,,...|   null|    1503273600|    true|null|
|B00006IAHD|                null|    5.0|                null|04 20, 2017|A1ROVL4AWYM9D6|    Amazon Customer|[,  Silver,,,,,,,...|   null|    1492646400|   false|null|
|B00006IAGW|                null|    5.0|Perfect! Used the...| 05 6, 2017| AV6UUM16WE8RE|             Amanda|                null|   null|    1494028800|    tru

In [14]:
data.filter(" summary is null").count()

957

In [15]:
data.filter(" reviewText is null").count()

2530

In [16]:
data.count()

2875917

In [17]:
data.filter(" reviewText is not null").count()

2873387

**Removing the Null ReviewText**

In [0]:
NoNullData=data.dropna(thresh=None,subset=('reviewText'))

In [19]:
NoNullData.filter(" reviewText is null").count()

0

In [20]:
NoNullData.show()

+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|               style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|0449819906| null|    5.0|I've read this bo...| 11 6, 2013|A3U4E9PIZ8OWH1|      Shirley Curtis|[,,,,,,  Kindle E...|   A  WONDERFUL BOOK|    1383696000|    true|   4|
|0449819906| null|    5.0|Nicely written di...|02 28, 2017|A3945D2TJ0PI86|            M. Smith|[,,,,,,  Kindle E...|                Nice|    1488240000|    true|null|
|0449819906| null|    5.0|             love it|02 25, 2017|A2WZK72HLQ7SPT|     Amazon Customer|[,,,,,,  Paperbac...|          Five Stars|    1487980800|    true|null


**Removing Punctuations and lowercasing**




In [0]:
from pyspark.sql import SQLContext
sqlContext = SQLContext

In [0]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType


In [0]:
data = data.select('overall', 'reviewText', 'verified')

In [83]:
data.show()

+-------+--------------------+--------+
|overall|          reviewText|verified|
+-------+--------------------+--------+
|    5.0|I've read this bo...|    true|
|    5.0|Nicely written di...|    true|
|    5.0|             love it|    true|
|    5.0|Good additional k...|    true|
|    5.0|A gazillion patte...|    true|
|    2.0|Just ok. Read bet...|    true|
|    5.0|The best knitting...|    true|
|    5.0|This book is a mo...|    true|
|    5.0|excellent variety...|    true|
|    5.0|Another winner M....|    true|
|    5.0|Love all the patt...|    true|
|    4.0|Good selection of...|    true|
|    4.0|Contains some int...|    true|
|    5.0|Super useful for ...|    true|
|    5.0|Loved it. Differe...|   false|
|    5.0|I love the book b...|    true|
|    5.0|Awesome book!!! A...|    true|
|    5.0|   Lots of stitches.|    true|
|    5.0|Easy to follow al...|    true|
|    5.0|I'm a fairly expe...|    true|
+-------+--------------------+--------+
only showing top 20 rows



In [0]:
data_clean = data

In [85]:
data_clean.show()

+-------+--------------------+--------+
|overall|          reviewText|verified|
+-------+--------------------+--------+
|    5.0|I've read this bo...|    true|
|    5.0|Nicely written di...|    true|
|    5.0|             love it|    true|
|    5.0|Good additional k...|    true|
|    5.0|A gazillion patte...|    true|
|    2.0|Just ok. Read bet...|    true|
|    5.0|The best knitting...|    true|
|    5.0|This book is a mo...|    true|
|    5.0|excellent variety...|    true|
|    5.0|Another winner M....|    true|
|    5.0|Love all the patt...|    true|
|    4.0|Good selection of...|    true|
|    4.0|Contains some int...|    true|
|    5.0|Super useful for ...|    true|
|    5.0|Loved it. Differe...|   false|
|    5.0|I love the book b...|    true|
|    5.0|Awesome book!!! A...|    true|
|    5.0|   Lots of stitches.|    true|
|    5.0|Easy to follow al...|    true|
|    5.0|I'm a fairly expe...|    true|
+-------+--------------------+--------+
only showing top 20 rows



Finding Word Length in the document

In [0]:
data_clean = data_clean.select(trim(lower(regexp_replace(col('reviewText'), '[^A-Za-z0-9 ]', ''))).alias('sentence'))

In [87]:
data_clean.show()

+--------------------+
|            sentence|
+--------------------+
|ive read this boo...|
|nicely written di...|
|             love it|
|good additional k...|
|a gazillion patte...|
|just ok read bett...|
|the best knitting...|
|this book is a mo...|
|excellent variety...|
|another winner m ...|
|love all the patt...|
|good selection of...|
|contains some int...|
|super useful for ...|
|loved it differen...|
|i love the book b...|
|awesome book a mu...|
|    lots of stitches|
|easy to follow al...|
|im a fairly exper...|
+--------------------+
only showing top 20 rows



In [0]:
data_clean =data_clean.withColumn("Length", size(split(data_clean['sentence'], ' ')))

In [89]:
data_clean.show()

+--------------------+------+
|            sentence|Length|
+--------------------+------+
|ive read this boo...|    76|
|nicely written di...|     3|
|             love it|     2|
|good additional k...|    45|
|a gazillion patte...|    53|
|just ok read bett...|     6|
|the best knitting...|    24|
|this book is a mo...|    33|
|excellent variety...|     5|
|another winner m ...|     4|
|love all the patt...|    16|
|good selection of...|    27|
|contains some int...|     4|
|super useful for ...|    31|
|loved it differen...|     7|
|i love the book b...|    11|
|awesome book a mu...|     5|
|    lots of stitches|     3|
|easy to follow al...|    17|
|im a fairly exper...|   371|
+--------------------+------+
only showing top 20 rows



**TOKENIZATION**

In [90]:
tokenizer = Tokenizer(inputCol="sentence", outputCol='tokens')
data_tokens = tokenizer.transform(data_clean).select('tokens')
data_tokens.show(5)

+--------------------+
|              tokens|
+--------------------+
|[ive, read, this,...|
|[nicely, written,...|
|          [love, it]|
|[good, additional...|
|[a, gazillion, pa...|
+--------------------+
only showing top 5 rows



**REMOVE STOP WORDS**

In [91]:
remover = StopWordsRemover(inputCol='tokens', outputCol='words_clean')
data_tokens = remover.transform(data_tokens).select('words_clean')
data_tokens.show(5)

+--------------------+
|         words_clean|
+--------------------+
|[ive, read, book,...|
|[nicely, written,...|
|              [love]|
|[good, additional...|
|[gazillion, patte...|
+--------------------+
only showing top 5 rows



In [92]:
pip install nltk

In [0]:
from pyspark.sql.functions import col, lit
from functools import reduce
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud 
import pandas as pd
import re
import string

In [94]:
data_tokens.withColumn("doc_id", monotonically_increasing_id())

DataFrame[words_clean: array<string>, doc_id: bigint]